# San Francisco Rental Prices Dashboard

In this notebook, you will compile the visualizations from the previous analysis into functions that can be used for a Panel dashboard.

In [52]:
# imports
import panel as pn
pn.extension('plotly')
import plotly.express as px
import pandas as pd
import hvplot.pandas
import matplotlib.pyplot as plt
import os
from pathlib import Path
from dotenv import load_dotenv

In [53]:
# Read the Mapbox API key
load_dotenv()
map_box_api = os.getenv("mapbox")
px.set_mapbox_access_token(map_box_api)

# Import Data

In [54]:
# Import the necessary CSVs to Pandas DataFrames
# YOUR CODE HERE!
file_path = Path("Data/sfo_neighborhoods_census_data.csv")
sfo_data = pd.read_csv(file_path, index_col="year")

nh_path = Path('Data/neighborhoods_coordinates.csv')
nh_df = pd.read_csv(nh_path)


In [55]:
# Housing Units per Year
# Rental Analysis code
grouped_df = sfo_data['housing_units']
housing_frame = grouped_df.to_frame()
units_avg_df = grouped_df.groupby(by = 'year').mean()
units_by_year_df = units_avg_df.reset_index()

# Average Gross Rent and Average Sales Prices
grouped_df_2 = sfo_data[['sale_price_sqr_foot','gross_rent']]
sales_avg_df = grouped_df_2.groupby(by = 'year').mean()
sales_rent_df = sales_avg_df.reset_index()

# Average Price by Neighborhood
sales_prices = sfo_data.groupby(by = ['year','neighborhood']).mean()
sales_prices.drop(['housing_units','gross_rent'], axis = 1, inplace = True)

# Most Expensive Neighborhoods
hood_mean_df = sfo_data.groupby(by = 'neighborhood').mean()
sorted_hood_df = hood_mean_df.sort_values(by = 'sale_price_sqr_foot', ascending = False)
high_price_slice = sorted_hood_df[0:9]
high_price_df = high_price_slice.reset_index()

# Parallel Categories
high_price_df_reset = high_price_df.reset_index()

# Neighborhood Map Data
neighborhood_df = sorted_hood_df.reset_index()
neighborhood_df.sort_values(by = 'neighborhood', ascending = True, inplace=True)
neighborhood_df.reset_index(inplace=True)
neighborhood_df.drop(['index'], axis=1, inplace=True)
concat_df = pd.concat([nh_df,neighborhood_df], axis = 'columns', join = 'outer')
concat_df.drop(['neighborhood'], axis=1, inplace = True)

grouped_df_2 = sfo_data[['sale_price_sqr_foot','gross_rent']]
sales_avg_df = grouped_df_2.groupby(by = 'year').mean()
sales_rent_df = sales_avg_df.reset_index()

- - -

## Panel Visualizations

In this section, you will copy the code for each plot type from your analysis notebook and place it into separate functions that Panel can use to create panes for the dashboard. 

These functions will convert the plot object to a Panel pane.

Be sure to include any DataFrame transformation/manipulation code required along with the plotting code.

Return a Panel pane object from each function that can be used to build the dashboard.

Note: Remove any `.show()` lines from the code. We want to return the plots instead of showing them. The Panel dashboard will then display the plots.

In [56]:
# Define Panel Visualization Functions
def housing_units_per_year():
    """Housing Units Per Year."""
    
    # YOUR CODE HERE!
    housing_plot = px.bar(
        units_by_year_df,
        x = 'year',
        y = 'housing_units',
        title = 'Housing Units Per Year'
    )
    


def average_gross_rent():
    """Average Gross Rent in San Francisco Per Year."""
    
    # YOUR CODE HERE!
    rent_plot = px.line(
        sales_rent_df,
        x = 'year',
        y = 'gross_rent', 
        title = 'Average Gross Rent in San Francisco Per Year'
    )



def average_sales_price():
    """Average Sales Price Per Year."""
    
    # YOUR CODE HERE!
    sales_plot = px.line(
        sales_rent_df,
        x = 'year',
        y = 'sale_price_sqr_foot',
        title = 'Average Sales Price Per Year'
    )

    return sales_plot



def average_price_by_neighborhood():
    """Average Prices by Neighborhood."""
    
    # YOUR CODE HERE!
    neighborhood_plot = sales_prices.hvplot(
        x = 'year', 
        y = 'sale_price_sqr_foot', 
        groupby = 'neighborhood',
        title = 'Average Prices by Neighborhood'
    )
    
    return neighborhood_plot



def top_most_expensive_neighborhoods():
    """Top 10 Most Expensive Neighborhoods."""

    # YOUR CODE HERE!
    expensive_plot = px.bar(
        high_price_df,
        x = 'neighborhood',
        y = 'sale_price_sqr_foot',
        title = 'Top 10 Most Expensive Neighborhoods'
    )

    return expensive_plot


def most_expensive_neighborhoods_rent_sales():
    """Comparison of Rent and Sales Prices of Most Expensive Neighborhoods."""   
    
    # YOUR CODE HERE!
    expensive_rent_sale_plot = high_price_df.hvplot.bar(
                                                        x='year',
                                                        xlabel='neighborhood',
                                                        y=('gross_rent','sale_price_sqr_foot'),
                                                        ylabel='Number of Housing Units',
                                                        groupby='neighborhood',
                                                        rot=90,
                                                        title=('Top 10 Expensive Neighborhoods in SFO'))
    return expensive_rent_sale_plot

    
def parallel_coordinates():
    """Parallel Coordinates Plot."""

    # YOUR CODE HERE!
    p1_plot = px.parallel_coordinates(
        high_price_df, 
        color = 'sale_price_sqr_foot',
        title = 'Parallel Coordinates Plot'
    )
    
    return p1_plot
    



def parallel_categories():
    """Parallel Categories Plot."""
    
    # YOUR CODE HERE!
    p2_plot = px.parallel_categories(
        high_price_df_reset,
        color = 'sale_price_sqr_foot',
        title = 'Parallel Categories Plot'
    )
    
    return p2_plot



def neighborhood_map():
    """Neighborhood Map."""

    # YOUR CODE HERE!
    map = px.scatter_mapbox(
                            concat_df,
                            lat='Lat',
                            lon='Lon',
                            size='sale_price_sqr_foot',
                            color='gross_rent',
                            zoom=10)
    pane_map = pn.pane.Plotly(map)

    return pane_map


def sunburst():
    """Sunburst Plot."""
    
    # YOUR CODE HERE!
    sunburst_plot = px.sunburst(most_expensive_neighborhoods_rent_sales,
                                path=['year', 'neighborhood'],
                                color='gross_rent',
                                color_continuous_scale='blues',
                                height=700,
                                width=700)

    return sunburst_plot


## Panel Dashboard

In this section, you will combine all of the plots into a single dashboard view using Panel. Be creative with your dashboard design!

In [57]:
# Create a Title for the Dashboard
# YOUR CODE HERE!
title = pn.Column('Real Estate Analysis for San Francisco 2010-2016')

# Create a tab layout for the dashboard
# YOUR CODE HERE!
first_column = pn.Column(
    "## Static Plots", 
    housing_units_per_year(),
    average_gross_rent(),
    average_sales_price(),
    top_most_expensive_neighborhoods()
    
)

second_column = pn.Column(
    "## Interactive Plots",
    average_price_by_neighborhood(),
    parallel_coordinates(),
    parallel_categories(),
    neighborhood_map()
)


# Create the dashboard
# YOUR CODE HERE!
dashboard = pn.Tabs(
    ("Static Plots", first_column), ("Interactive Plots", second_column)
)


## Serve the Panel Dashboard

In [58]:
# Serve the# dashboard
# YOUR CODE HERE!
dashboard.servable()

BokehModel(combine_events=True, render_bundle={'docs_json': {'70c8d564-adb3-4365-9807-4b784f7b7635': {'defs': …

Tabs
    [0] Column
        [0] Markdown(str)
        [1] Str(None)
        [2] Str(None)
        [3] Plotly(Figure)
        [4] Plotly(Figure)
    [1] Column
        [0] Markdown(str)
        [1] Row
            [0] HoloViews(DynamicMap)
            [1] Column
                [0] WidgetBox
                    [0] Select(margin=(20, 20, 20, 20), name='neighborhood', options=['Alamo Square', ...], value='Alamo Square', width=250)
                [1] VSpacer()
        [2] Plotly(Figure, visible=False)
        [3] Plotly(Figure, visible=False)
        [4] Plotly(Figure, visible=False)

# Debugging

Note: Some of the Plotly express plots may not render in the notebook through the panel functions.

However, you can test each plot by uncommenting the following code

In [59]:
# housing_units_per_year()
housing_units_per_year

<function __main__.housing_units_per_year()>

In [60]:
# average_gross_rent()
average_gross_rent

<function __main__.average_gross_rent()>

In [61]:
# average_sales_price()
average_sales_price

<function __main__.average_sales_price()>

In [62]:
# average_price_by_neighborhood()
average_price_by_neighborhood

<function __main__.average_price_by_neighborhood()>

In [63]:
# top_most_expensive_neighborhoods()
top_most_expensive_neighborhoods

<function __main__.top_most_expensive_neighborhoods()>

In [64]:
# most_expensive_neighborhoods_rent_sales()
most_expensive_neighborhoods_rent_sales

<function __main__.most_expensive_neighborhoods_rent_sales()>

In [65]:
# neighborhood_map().show()
neighborhood_map

<function __main__.neighborhood_map()>

In [66]:
# parallel_categories()
parallel_coordinates

<function __main__.parallel_coordinates()>

In [67]:
# parallel_coordinates()
parallel_coordinates

<function __main__.parallel_coordinates()>

In [68]:
# sunburst()
sunburst

<function __main__.sunburst()>